In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
#число хромосом в популяции 
K = 50

In [3]:
# функция выживаемости
def fitness_func(chromosome): 
    if np.abs(chromosome[0] + 2 * chromosome[1] + 3 * chromosome[2] + 4 * chromosome[3] - 30) == 0: 
        return -1
    else: 
        return 1/np.abs(chromosome[0] + 2 * chromosome[1] + 3 * chromosome[2] + 4 * chromosome[3] - 30)

In [4]:
# функция нахождения пар мама-папа
def find_parents(data, probabilities):
    #список для хранения индексов массива, np.random.choice на вход принимает 1-D массив, поэтому 
    #будем случайно выбирать не саму строку массива, а индекс
    index_list = np.arange(len(data))
    #список для хранения индекса мамы и папы
    parents = [] 
    parents_count = 0 
    while parents_count < K: 
        mother = np.random.choice(index_list, 1, p=probabilities)[0]
        father = np.random.choice(index_list, 1, p=probabilities)[0]
        if mother != father and not [mother,father] in parents: 
            parents.append([mother,father])
            parents_count += 1
    return parents  

In [11]:
#функция скрещивания
def breeding_function (data, parents):
    descendants = np.empty((0,4), int)
    splitting_point = 0 
    for couples in parents:
        descedand = []
        if splitting_point == 4: 
            splitting_point = 0
        splitting_point += 1
        descendant = np.concatenate((data[couples[0]][:splitting_point], data[couples[1]][splitting_point:]), axis=0)
        descendants = np.append(descendants,np.array([descendant]),axis = 0)
    descendants_fitness = [fitness_func(chromosome) for chromosome in descendants]
    return descendants, descendants_fitness

In [6]:
#функция мутации
def mutation(descendants,descendants_fitness ):
    # Находим самого нежизнеспособного потомка
    min_descendants_fitness = descendants_fitness[0]
    min_descendants_fitness_index = 0
    for i in range(1,len(descendants_fitness)): 
        if descendants_fitness[i] < min_descendants_fitness: 
            min_descendants_fitness = descendants_fitness[i]
            min_descendants_fitness_index = i 
    mutant = descendants[min_descendants_fitness_index]
    # меняем пока коэффициент выживаемости не станет больше
    iteration = 0 
    while iteration < 100: 
        mutant_chromosome = np.random.randint(1, 21+1, (1,4))
        iteration += 1
        if fitness_func(mutant_chromosome[0]) > min_descendants_fitness:
            break
    descendants = np.delete(descendants, (min_descendants_fitness_index), axis = 0)
    descendants_fitness = np.delete(descendants_fitness, min_descendants_fitness_index, 0)
    descendants = np.append(descendants, [mutant_chromosome[0]], axis = 0)
    descendants_fitness = np.append(descendants_fitness, fitness_func(mutant_chromosome[0]))
    return descendants, descendants_fitness

In [7]:
#функция естественного отбора
def find_strongest(data, fitness, descendants, descendants_fitness):
    data = np.append(data, descendants, axis = 0)
    fitness = np.append(fitness, descendants_fitness, axis = 0)
    data_fitness = np.column_stack([data, fitness])
    data_fitness = data_fitness[data_fitness[:, data_fitness.shape[1]-1].argsort()][::-1][:K]
    return data_fitness[:,:data_fitness.shape[1]-1], data_fitness[:,data_fitness.shape[1]-1]

In [8]:
#проверка, что решение найдено 
def solution_exist(data, fitness): 
    answer = []
    for i in range(len(fitness)):
        if fitness[i] == -1: 
            answer.append(data[i])
    return answer     

In [15]:
data = np.random.randint(1, 21+1, (K,4))
fitness = [fitness_func(chromosome) for chromosome in data]
while True: 
    answer = solution_exist(data, fitness)
    if len(answer) != 0: 
        break 
    probabilities = [fit/np.sum(fitness) for fit in fitness]
    parents = find_parents(data, probabilities)
    descendants, descendants_fitness = breeding_function (data, parents)
    answer = solution_exist(descendants, descendants_fitness)
    if len(answer) != 0: 
        break 
    descendants, descendants_fitness = mutation(descendants,descendants_fitness)
    answer = solution_exist(descendants, descendants_fitness)
    if len(answer) != 0: 
        break
    data,fitness = find_strongest(data, fitness, descendants, descendants_fitness)    
print(answer)


[array([4., 8., 2., 1.]), array([4., 8., 2., 1.])]
